In [35]:
!pip install transformers==4.10.0

  Using cached transformers-4.10.0-py3-none-any.whl.metadata (51 kB)
  Using cached tokenizers-0.10.3-cp39-cp39-win_amd64.whl.metadata (5.8 kB)
Using cached transformers-4.10.0-py3-none-any.whl (2.8 MB)
Using cached tokenizers-0.10.3-cp39-cp39-win_amd64.whl (2.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.0.0 requires transformers<5.0.0,>=4.34.0, but you have transformers 4.10.0 which is incompatible.


In [1]:
import sys
print(sys.version)

3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]


In [2]:
pip show transformers

Name: transformers
Version: 4.10.0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: d:\anaconda\envs\py39\lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, sacremoses, tokenizers, tqdm
Required-by: sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import re
import demoji

# Load dataset
df = pd.read_csv("D:\\Sinophobia Sentiment Analysis\\archive\\Tweets_UK\\Tweets_UK.csv", low_memory=False)


In [4]:
df

,Unnamed: 0,created_at,id,text,user_location
0,9,Sun Jan 02 04:35:30 +0000 2022,1477498765485375489,We can only hope https://t.co/B4KkrXivxa,"New Westminster, BC"
1,23,Sun Jan 02 04:35:28 +0000 2022,1477498757126266884,RT @Azodoh: @EricTopol @NEJM Is the protection...,United Kingdom
2,30,Sun Jan 02 04:35:30 +0000 2022,1477498761605697536,"@firedad065 I understand this, this is where I...","Battle Creek, MI"
3,74,Sun Jan 02 04:35:33 +0000 2022,1477498775761473542,"RT @WWECreative_ish: ""Our roster is decimated ...","Derby, England"
4,99,Sun Jan 02 04:35:34 +0000 2022,1477498781880963073,RT @anilkjain61: Don’t dismiss Omicron as ‘mil...,"Manchester, Northwest, UK"
...,...,...,...,...,...
16958861,91217,Fri Aug 21 22:50:50 +0000 2020,1296942867894669313,Britain could go into a SECOND national lockdo...,"London, UK"
16958862,91441,Fri Aug 21 22:50:56 +0000 2020,1296942895199649792,@marcorandazza Do you have to wear a mask in t...,UK
16958863,91489,Fri Aug 21 22:50:58 +0000 2020,1296942901478535168,RT @LaylaMoran: I've written to the Prime Mini...,"Manchester, UK"
16958864,91550,Fri Aug 21 22:50:59 +0000 2020,1296942904229888000,RT @ONS: #COVID19 was the 8th most frequent un...,"Colchester, UK"


In [5]:
# Data cleaning
df = df.drop(columns=['Unnamed: 0', 'id', 'user_location'])

df = df.dropna(axis=0)


In [6]:
df

,created_at,text
0,Sun Jan 02 04:35:30 +0000 2022,We can only hope https://t.co/B4KkrXivxa
1,Sun Jan 02 04:35:28 +0000 2022,RT @Azodoh: @EricTopol @NEJM Is the protection...
2,Sun Jan 02 04:35:30 +0000 2022,"@firedad065 I understand this, this is where I..."
3,Sun Jan 02 04:35:33 +0000 2022,"RT @WWECreative_ish: ""Our roster is decimated ..."
4,Sun Jan 02 04:35:34 +0000 2022,RT @anilkjain61: Don’t dismiss Omicron as ‘mil...
...,...,...
16958861,Fri Aug 21 22:50:50 +0000 2020,Britain could go into a SECOND national lockdo...
16958862,Fri Aug 21 22:50:56 +0000 2020,@marcorandazza Do you have to wear a mask in t...
16958863,Fri Aug 21 22:50:58 +0000 2020,RT @LaylaMoran: I've written to the Prime Mini...
16958864,Fri Aug 21 22:50:59 +0000 2020,RT @ONS: #COVID19 was the 8th most frequent un...


In [7]:
final = pd.DataFrame()

dfs_to_concat = []

# Filtering
keywords = ['china', 'chinese', 'sinophobia', 'wuhan', 'kung flu', 'cn', 'ccp', 'yellow peril', 'chink', ' chinks', 'chingchong', 'ching chong', 'gook', ' chyna',
           'mainland', ' mainlander', 'bugland', 'chines', 'mainla', 'chinazi', 'bugmen', 'chankoro', 'insectoid']

for keyword in keywords:
    filtered_df = df[df['text'].str.contains(keyword, case=False, na=False)]
    if not filtered_df.empty:
        dfs_to_concat.append(filtered_df)

# Concatenate all filtered DataFrames
if dfs_to_concat:
    final = pd.concat(dfs_to_concat, ignore_index=True)

final = final.drop_duplicates(subset=['text'])

final.head()

,created_at,text
0,Sun Jan 02 04:46:25 +0000 2022,RT @AlanRMacLeod: New York has 10x as many COV...
1,Sun Jan 02 04:49:52 +0000 2022,RT @globaltimesnews: Police in Central China's...
2,Sun Jan 02 04:54:02 +0000 2022,"Regrettably, your bollocks is exposed as bollo..."
3,Sun Jan 02 05:00:05 +0000 2022,"As trade stalls, Vietnamese urges China to reo..."
4,Sun Jan 02 05:01:28 +0000 2022,RT @JeremiahBilas: @AlanRMacLeod Many still re...


In [8]:
len(final)

164799

In [9]:
df_withtime = final

In [10]:
final = final.drop(columns=['created_at'])

In [11]:
final.head()

,text
0,RT @AlanRMacLeod: New York has 10x as many COV...
1,RT @globaltimesnews: Police in Central China's...
2,"Regrettably, your bollocks is exposed as bollo..."
3,"As trade stalls, Vietnamese urges China to reo..."
4,RT @JeremiahBilas: @AlanRMacLeod Many still re...


In [12]:
len(final)

164799

In [13]:
final.to_csv("D:\\Sinophobia Sentiment Analysis\\UKAfterFilter.csv")

In [14]:
contractionsWithAnotherInvertedComma = {
"ain’t": "am not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
"couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
"hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "he’s": "he is",
"how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
"i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
"it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us",
"ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
"mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have",
"shan’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
"she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have",
"so’ve": "so have", "so’s": "so is", "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
"there’d’ve": "there would have", "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
"we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have",
"who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
"would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have",
"y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
"you’re": "you are", "you’ve": "you have"
}

In [15]:
contractions = {
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not",
"couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
"i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
"shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she shall / she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
"so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
"y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"
}

In [16]:
abbreviations_dict = {
    "btw": "by the way", "brb": "be right back",
    "b4n": "bye for now",
    "fyi": "for your information",
    "ilu": "i love you",
    "iow": "in other words",
    "roflol": "rolling on the floor laughing out loud",
    "rotflmao": "rolling on the floor laughing my ass off",
    "fb": "facebook", "ig": "instagram", "li": "linkedin", "yt": "youtube", "tw": "twitter",
    "dm": "direct message", "mt": "modified tweet", "pm": "private message", "rt": "retweet",
    "b2b": "business to business", "b2c": "business to consumer", "cmgr": "community manager",
    "cms": "content management system", "cpc": "cost per click", "cpm": "cost per thousand impressions",
    "cr": "conversion rate", "cro": "conversion rate optimization", "cta": "call to action",
    "ctr": "click-through rate", "roi": "return on investment", "smb": "small and midsize businesses",
    "smp": "social media platform", "smm": "social media marketing", "smo": "social media optimization",
    "solomo": "social, local and mobile", "srp": "social relationship platform", "tos": "terms of service",
    "ugc": "user-generated content", "api": "application programming interface", "cx": "customer experience",
    "esp": "email service provider", "ga": "google analytics", "isp": "internet service provider",
    "pv": "page views", "rss": "really simple syndication", "saas": "software as a service",
    "sem": "search engine marketing", "seo": "search engine optimization", "sov": "share of voice",
    "ui": "user interface", "url": "uniform resource locator", "uv": "unique views", "ux": "user experience",
    "afaik": "as far as i know", "ama": "ask me anything", "btaim": "be that as it may",
    "bts": "behind the scenes", "dae": "does anyone else", "dyk": "did you know",
    "eli5": "explain like i’m five", "fbf": "flashback friday", "fbo": "facebook official",
    "ff": "follow friday", "fomo": "fear of missing out", "ftfy": "fixed that for you",
    "ftw": "for the win", "g2g": "got to go", "gg": "good game", "gtr": "got to run",
    "hbd": "happy birthday", "hifw": "how i feel when", "hmb": "hit me back", "hmu": "hit me up",
    "ht": "hat tip", "h/t": "hat tip", "hth": "here to help", "icymi": "in case you missed it",
    "idc": "i don’t care", "idk": "i don’t know", "ikr": "i know, right?", "ily": "i love you",
    "imho": "in my humble opinion", "imo": "in my opinion", "irl": "in real life", "jk": "just kidding",
    "lmao": "laughing my ass off", "lmk": "let me know", "lms": "like my status",
    "lol": "laughing out loud", "mcm": "man crush monday", "mfw": "my face when",
    "mtfbwy": "may the force be with you", "nbd": "no big deal", "nm": "not much",
    "nsfw": "not safe for work", "nvm": "never mind", "oh": "overheard", "omw": "on my way",
    "ootd": "outfit of the day", "op": "original poster", "otp": "one true pairing", "ppl": "people",
    "rofl": "rolling on the floor laughing", "roflmao": "rolling on the floor laughing my ass off",
    "sfw": "safe for work", "smh": "shaking my head", "tbh": "to be honest", "tbbh": "to be brutally honest",
    "tbt": "throwback thursday", "tfw": "that feeling when", "tgif": "thank god it’s friday",
    "til": "today i learned", "tl;dr": "too long; didn’t read", "tmi": "too much information",
    "wbu": "what about you?", "wbw": "way back wednesday", "wfh": "work from home", "yolo": "you only live once",
    "afk": "away from keyboard", "asap": "as soon as possible", "atk": "at the keyboard",
    "atm": "at the moment", "a3": "anytime, anywhere, anyplace", "bak": "back at keyboard",
    "bbl": "be back later", "bbs": "be back soon", "bfn": "bye for now", "brt": "be right there",
    "b4": "before", "cu": "see you", "cul8r": "see you later", "cya": "see you", "faq": "frequently asked questions",
    "fc": "fingers crossed", "fwiw": "for what it's worth", "gal": "get a life", "gn": "good night",
    "gmta": "great minds think alike", "gr8": "great!", "g9": "genius", "ic": "i see", "icq": "i seek you",
    "kiss": "keep it simple, stupid", "ldr": "long distance relationship", "ltns": "long time no see",
    "l8r": "later", "mte": "my thoughts exactly", "m8": "mate", "nrn": "no reply necessary", "oic": "oh i see",
    "pita": "pain in the a..", "prt": "party", "prw": "parents are watching", "qpsa": "que pasa?",
    "sk8": "skate", "stats": "your sex and age", "asl": "age, sex, location", "thx": "thank you",
    "ttfn": "ta-ta for now!", "ttyl": "talk to you later", "u": "you", "u2": "you too", "u4e": "yours for ever",
    "wb": "welcome back", "wtf": "what the f...", "wtg": "way to go!", "wuf": "where are you from?",
    "w8": "wait...", "7k": "sick:-d laugher"
}


In [17]:
# Preprocessing
class preprocess():
    def __init__(self, df, contractions, otherContractions):
        self.df = df
        self.contractions = contractions
        self.otherContractions = otherContractions

    def lower(self, tweet):
        return tweet.lower()

    def abbreviate(self, tweet):
        tweet_tokens = tweet.split(' ')
        for i, token in enumerate(tweet_tokens):
            cleaned_token = re.sub('[^a-zA-Z0-9-_.]', '', token)
            if cleaned_token.lower() in abbreviations_dict:
                tweet_tokens[i] = abbreviations_dict[cleaned_token.lower()]
        return ' '.join(tweet_tokens)

    def expand(self, tweet):
        for word in tweet.split():
            if word in self.contractions.keys():
                tweet = tweet.replace(word, self.contractions[word])
            elif word in self.otherContractions.keys():
                tweet = tweet.replace(word, self.otherContractions[word])
        return tweet

    def emoji2text(self, tweet):
        emojis = demoji.findall(tweet)
        new_tweet = []
        for word in tweet.split():
            if word in emojis.keys():
                tweet = tweet.replace(word, emojis[word])
                new_tweet.append(emojis[word])
            wordmojis = demoji.findall(word)
            for char in word:
                if char in wordmojis.keys():
                    tweet = tweet.replace(word, wordmojis[char])

        return tweet

    def remove_hashtags(self, tweet):
        return re.sub(r'\#w+', '', tweet)

    def remove_mentions(self, tweet):
        for word in tweet.split():
            if word[0] == '@':
                tweet = tweet.replace(word, '')
        return tweet

    def remove_punctuations(self, tweet):
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')
        return tweet.translate(trantab)

    def remove_url(self, tweet):
        return re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet, flags = re.MULTILINE)

    def preprocess_tweet(self, tweet):
        tweet = self.lower(tweet)
        tweet = self.abbreviate(tweet)
        tweet = self.expand(tweet)
        tweet = self.emoji2text(tweet)
        tweet = self.remove_mentions(tweet)
        tweet = self.remove_url(tweet)
        tweet = self.remove_hashtags(tweet)
        tweet = self.remove_punctuations(tweet)

        return tweet

In [18]:
import csv
import string
pp_class = preprocess(final, contractions, contractionsWithAnotherInvertedComma)
final['text'] = final['text'].apply(lambda x : pp_class.preprocess_tweet(x))

In [19]:
def bruteGen(tweet):
    tweet = tweet.replace("indiavscorona", "india versus coronavirus")
    tweet = tweet.replace("outbreakindia", "outbreak india")
    tweet = tweet.replace("real”", "real")
    tweet = tweet.replace("mutra", "urine")
    tweet = tweet.replace("fakenews", "fake news")
    tweet = tweet.replace("“omg", "oh my god")
    tweet = tweet.replace("“damn", "damn")
    tweet = tweet.replace("god’s", "gods")
    tweet = tweet.replace("lockdownextension", "lockdown extension")
    tweet = tweet.replace("कोरोना", "coronavirus")
    tweet = tweet.replace("indiathanks", "india thanks")
    tweet = tweet.replace("coronacoronavirus", "coronavirus")
    tweet = tweet.replace('coronavirusinsa', "coronavirus in south africa")
    tweet = tweet.replace('coronaviruscanada', 'coronavirus canada')
    tweet = tweet.replace('coronavirusau', 'coronavirus australia')
    tweet = tweet.replace('coronavirusaus', 'coronavirus australia')
    tweet = tweet.replace('cuomoprimetime', 'new york governor prime time')
    tweet = tweet.replace('letsfightcoronavirus', 'let us fight coronavirus')
    tweet = tweet.replace("covid19", "coronavirus")
    tweet = tweet.replace("covid", "coronavirus")
    tweet = tweet.replace("aprilfoolsday", "april fools day")
    tweet = tweet.replace("covidー19", "coronavirus")
    tweet = tweet.replace("stayathome", "stay at home")
    tweet = tweet.replace("“april", "april")
    tweet = tweet.replace("“i", "i")
    tweet = tweet.replace("aprilfools", "april fools")
    tweet = tweet.replace("coronavirusoutbreak", "coronavirus outbreak")
    tweet = tweet.replace("virusー19", "coronavirus")
    tweet = tweet.replace("fool’s", "fools")
    tweet = tweet.replace("what’s", "what is")
    tweet = tweet.replace("coronavirus”", "coronavirus")
    tweet = tweet.replace("fools”", "fools")
    tweet = tweet.replace("stayhome", "stay home")
    tweet = tweet.replace("quarantinelife", "quarantine life")
    tweet = tweet.replace("tablighijamaat", "muslims")
    tweet = tweet.replace("corona”", "coronavirus")
    tweet = tweet.replace("fauci", "physician")
    tweet = tweet.replace("april’s", "april")
    tweet = tweet.replace("pmkcallscurfewextension", "prime minister calls for curfew extension")
    tweet = tweet.replace("“virus", "coronavirus")
    tweet = tweet.replace("virus”", "coronavirus")
    tweet = tweet.replace("“corona", "coronavirus")
    tweet = tweet.replace("coronavirustruth", "coronavirus truth")
    tweet = tweet.replace("socialdistancing", "social distancing")
    tweet = tweet.replace("homestaysafe", "home stay safe")
    tweet = tweet.replace("“coronavirus", "coronavirus")
    tweet = tweet.replace("coronavirusupdate", "coronavirus update")
    tweet = tweet.replace("virusvirus", "coronavirus")
    tweet = tweet.replace("coronaviruspandemic", "coronavirus pandemic")
    tweet = tweet.replace("thelockdown", "the lockdown")
    tweet = tweet.replace("nizamuddin", "delhi")
    tweet = tweet.replace("trump’s", "donald trump")
    tweet = tweet.replace("“the", "the")
    tweet = tweet.replace("virus2019", "coronavirus")
    tweet = tweet.replace("indiafightscorona", "india fights coronavirus")
    tweet = tweet.replace("homesavelives", "home save lives")
    tweet = tweet.replace("everyone’s", "everyone")
    tweet = tweet.replace("coronariskforprisoners", "coronavirus risk for prisoners")
    tweet = tweet.replace("coronavirususa", "coronavirus usa")
    tweet = tweet.replace("tablighi", "mosque")
    tweet = tweet.replace("delhimarkaz", "delhi mosque")
    tweet = tweet.replace("coronajihad", "coronavirus struggle")
    tweet = tweet.replace("coronajihaad", "coronavirus struggle")
    tweet = tweet.replace("aprilfool", "april fool")
    tweet = tweet.replace("trumppressconference", "trump press conference")
    tweet = tweet.replace("i’m", "i am")
    tweet = tweet.replace("tigerking", "tiger king")
    tweet = tweet.replace("it’s", "it is")
    tweet = tweet.replace("trumpvirus", "trump virus")
    tweet = tweet.replace("today’s", "today is")
    tweet = tweet.replace("“you", "you")
    tweet = tweet.replace("“a", "a")
    tweet = tweet.replace("fools’", "fools")
    tweet = tweet.replace("rtgnews", "news")
    tweet = tweet.replace("19india", "india")
    tweet = tweet.replace("coronavirusindia", "coronavirus india")
    tweet = tweet.replace("y’all", "you all")
    tweet = tweet.replace("मीडिया", "media")
    tweet = tweet.replace("here’s", "here is")
    tweet = tweet.replace("“we", "we")
    tweet = tweet.replace("“fuck", "fuck")
    tweet = tweet.replace("flattenthecurve", "flatten the curve")
    tweet = tweet.replace("jammuandkashmir", "jammu and kashmir")
    tweet = tweet.replace("chriscuomo", "new york governor")
    tweet = tweet.replace("‘april", "april")
    tweet = tweet.replace("dranbumani", "doctor")
    tweet = tweet.replace("tndemandsmasstesting", "tamil nadu demands mass testing")
    tweet = tweet.replace("tabligi", "muslims")
    tweet = tweet.replace("don’t", "do not")
    tweet = tweet.replace("वायरस", "virus")
    tweet = tweet.replace("letsfightvirus", "let us fight virus")
    return tweet

In [20]:
final['text'] = final['text'].apply(lambda x : bruteGen(x))

In [21]:
final.head

<bound method NDFrame.head of                                                      text
0       retweet  new york has 10x as many coronavirus ...
1       retweet  police in central china s wuhan arres...
2       regrettably  your bollocks is exposed as bollo...
3       as trade stalls  vietnamese urges china to reo...
4       retweet   many still refuse to believe this  b...
...                                                   ...
377611  usps chief louis dejoy says he will not restor...
377617  usps chief louis dejoy says he will not restor...
378621  retweet  ⁣the inaugural helsinki biennial will...
378948  retweet  episode 117   if we cannot use the cl...
379191  retweet  ‘they do not want hongkongers’ – how ...

[164799 rows x 1 columns]>

In [22]:
final.to_csv("D:\\Sinophobia Sentiment Analysis\\UKAfterpreprocess.csv")

In [23]:
labels = ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic',
       'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Joking']

In [24]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [25]:
df = pd.DataFrame()
df['Tweet'] = final['text']
values = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] * len(final)
df[labels] = values

In [26]:
df.reset_index(drop=True, inplace=True)
df

,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,retweet new york has 10x as many coronavirus ...,0,0,0,0,0,0,0,0,0,0
1,retweet police in central china s wuhan arres...,0,0,0,0,0,0,0,0,0,0
2,regrettably your bollocks is exposed as bollo...,0,0,0,0,0,0,0,0,0,0
3,as trade stalls vietnamese urges china to reo...,0,0,0,0,0,0,0,0,0,0
4,retweet many still refuse to believe this b...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
164794,usps chief louis dejoy says he will not restor...,0,0,0,0,0,0,0,0,0,0
164795,usps chief louis dejoy says he will not restor...,0,0,0,0,0,0,0,0,0,0
164796,retweet ⁣the inaugural helsinki biennial will...,0,0,0,0,0,0,0,0,0,0
164797,retweet episode 117 if we cannot use the cl...,0,0,0,0,0,0,0,0,0,0


In [27]:
import torch
class CustomDataset(torch.utils.data.Dataset):
   def __init__(self,df,tokenizer,max_len):
     self.df = df
     self.tokenizer = tokenizer
     self.max_len = max_len
     self.title = self.df['Tweet']
     self.targets = self.df[labels].values

   def __len__(self):
     return len(self.title)

   def __getitem__(self,index):
     title = str(self.title[index])
     title = " ".join(title.split())

     inputs = self.tokenizer.encode_plus(
         title,
         None,
         add_special_tokens=True,
         max_length=self.max_len,
         padding='max_length',
         return_token_type_ids=True,
         truncation=True,
         return_attention_mask=True,
         return_tensors='pt'
     )
     return {
         'input_ids':inputs['input_ids'].flatten(),
         'attention_mask':inputs['attention_mask'].flatten(),
         'token_type_ids':inputs['token_type_ids'].flatten(),
         'targets':torch.FloatTensor(self.targets[index])
     }

In [28]:
MAX_LEN=256
_dataset = CustomDataset(df, tokenizer, MAX_LEN)
_data_loader = torch.utils.data.DataLoader(
    _dataset,
    shuffle=False,
    batch_size=8,
    num_workers=0
)

In [29]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [30]:
class BERT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = BertModel.from_pretrained('bert-base-uncased')
        self.layer2 = torch.nn.Dropout(0.3)
        self.layer3 = torch.nn.Linear(768, 10)

    def forward(self, ids, mask, token_type_ids, return_dict = False):
        unw, out_1 = self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[0], self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[1]
        out_2 = self.layer2(out_1)
        out_final = self.layer3(out_2)
        return out_final


model = BERT()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT(
  (layer1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [31]:
model = torch.load("D:\\Sinophobia Sentiment Analysis\\Transformers4.10Bertmodel2.pth")

In [32]:
model.eval()
bert_outputs = []
with torch.no_grad():

  for index, batch in enumerate(_data_loader):
         input_ids = batch['input_ids'].to(device, dtype=torch.long)
         attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
         token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
         targets =  batch['targets'].to(device, dtype=torch.float)
         outputs = model(input_ids, attention_mask, token_type_ids)
         bert_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

In [33]:
test_outputs = np.array(bert_outputs)

for i in range(test_outputs.shape[0]):
    for j in range(test_outputs.shape[1]):
        if test_outputs[i][j] >= 0.5: test_outputs[i][j] = 1
        else: test_outputs[i][j] = 0

In [34]:
df[labels] = test_outputs.astype(int)

In [35]:
df

,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,retweet new york has 10x as many coronavirus ...,0,0,0,0,0,0,0,0,1,0
1,retweet police in central china s wuhan arres...,0,0,0,0,0,0,0,0,1,0
2,regrettably your bollocks is exposed as bollo...,0,0,0,0,0,0,1,1,0,0
3,as trade stalls vietnamese urges china to reo...,0,0,0,0,0,0,0,0,1,0
4,retweet many still refuse to believe this b...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
164794,usps chief louis dejoy says he will not restor...,0,0,0,0,0,0,1,0,1,0
164795,usps chief louis dejoy says he will not restor...,0,0,0,0,0,0,0,0,1,0
164796,retweet ⁣the inaugural helsinki biennial will...,1,0,0,0,0,0,0,0,1,0
164797,retweet episode 117 if we cannot use the cl...,0,0,0,0,0,0,1,0,0,1


In [36]:
df_withtime.reset_index(drop=True, inplace=True)
df_withtime

,created_at,text
0,Sun Jan 02 04:46:25 +0000 2022,RT @AlanRMacLeod: New York has 10x as many COV...
1,Sun Jan 02 04:49:52 +0000 2022,RT @globaltimesnews: Police in Central China's...
2,Sun Jan 02 04:54:02 +0000 2022,"Regrettably, your bollocks is exposed as bollo..."
3,Sun Jan 02 05:00:05 +0000 2022,"As trade stalls, Vietnamese urges China to reo..."
4,Sun Jan 02 05:01:28 +0000 2022,RT @JeremiahBilas: @AlanRMacLeod Many still re...
...,...,...
164794,Fri Aug 21 16:57:32 +0000 2020,USPS chief Louis DeJoy says he won't restore m...
164795,Fri Aug 21 20:18:03 +0000 2020,USPS chief Louis DeJoy says he won't restore m...
164796,Wed Apr 07 08:26:45 +0000 2021,RT @HELbiennial: ⁣The inaugural Helsinki Bienn...
164797,Fri Nov 19 13:47:52 +0000 2021,"RT @PodGeeked: Episode 117: ""If we can’t use t..."


In [37]:
UK = pd.DataFrame()
UK['Tweet'] = df['Tweet']
UK['text'] = df_withtime['text']
UK['date'] = df_withtime['created_at']
UK[labels] = df[labels]
UK

,Tweet,text,date,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,retweet new york has 10x as many coronavirus ...,RT @AlanRMacLeod: New York has 10x as many COV...,Sun Jan 02 04:46:25 +0000 2022,0,0,0,0,0,0,0,0,1,0
1,retweet police in central china s wuhan arres...,RT @globaltimesnews: Police in Central China's...,Sun Jan 02 04:49:52 +0000 2022,0,0,0,0,0,0,0,0,1,0
2,regrettably your bollocks is exposed as bollo...,"Regrettably, your bollocks is exposed as bollo...",Sun Jan 02 04:54:02 +0000 2022,0,0,0,0,0,0,1,1,0,0
3,as trade stalls vietnamese urges china to reo...,"As trade stalls, Vietnamese urges China to reo...",Sun Jan 02 05:00:05 +0000 2022,0,0,0,0,0,0,0,0,1,0
4,retweet many still refuse to believe this b...,RT @JeremiahBilas: @AlanRMacLeod Many still re...,Sun Jan 02 05:01:28 +0000 2022,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
164794,usps chief louis dejoy says he will not restor...,USPS chief Louis DeJoy says he won't restore m...,Fri Aug 21 16:57:32 +0000 2020,0,0,0,0,0,0,1,0,1,0
164795,usps chief louis dejoy says he will not restor...,USPS chief Louis DeJoy says he won't restore m...,Fri Aug 21 20:18:03 +0000 2020,0,0,0,0,0,0,0,0,1,0
164796,retweet ⁣the inaugural helsinki biennial will...,RT @HELbiennial: ⁣The inaugural Helsinki Bienn...,Wed Apr 07 08:26:45 +0000 2021,1,0,0,0,0,0,0,0,1,0
164797,retweet episode 117 if we cannot use the cl...,"RT @PodGeeked: Episode 117: ""If we can’t use t...",Fri Nov 19 13:47:52 +0000 2021,0,0,0,0,0,0,1,0,0,1


In [38]:
UK.to_csv("D:\\Sinophobia Sentiment Analysis\\Results\\UK.csv")